In [1]:
from google_trans_new import google_translator
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

In [2]:
translator = google_translator()
#sentence_bert_model = np.load('model/embedding.npy')

# utiliser un modèle déjà entrainé
sentence_bert_model = SentenceTransformer('bert-base-nli-mean-tokens')

def get_df():
    path = 'data/data_final.csv'
    df = pd.read_csv(path, sep=';')
    q_vec_list = np.load('model/q_embedding.npy')
    a_vec_list = np.load('model/a_embedding.npy')
    q_df = pd.DataFrame(q_vec_list)
    a_df = pd.DataFrame(a_vec_list)
    return df, q_df, a_df

In [3]:
# lecture des fichiers avec pandas
def get_dataframe(path):
    df = pd.read_csv(path, sep=';', names=['Question', 'Type', 'Answer'])
    return df
# diviser les deux listes
def get_list(df):
    question_list = df['Question'].to_list()
    answer_list = df['Answer'].to_list()
    return question_list, answer_list

# diviser les trois types de questions
def classify_data(df):
    psycho_frame = df[df['Type'] == 'psychology']
    other_frame = df[df['Type'] == 'not_drug_question']
    drug_frame = df[(df['Type'] != 'psychology') & (
        df['Type'] != 'not_drug_question')]
    return psycho_frame, other_frame, drug_frame

# calculer la similarité entre deux vecteurs avec outil de pyTorch
def get_cosin_sim(vector_a, vector_b):
    #sim = 1 - cosine(vector_a, vector_b)
    return util.pytorch_cos_sim(vector_a, vector_b)

# retourner la maximum des similarités et son index dans une liste des phrases (questions/réponses)
def get_similarity_max(vector_a, liste):
    sim_list = []
    for ele in liste:
        #sim = cosine_similarity(vector_a, sentence_bert_model.encode([ele])[0])
        sim = get_cosin_sim(vector_a, sentence_bert_model.encode([ele])[0])
        sim_list.append(sim)
    max_sim = max(sim_list)
    idx = sim_list.index(max_sim)
    return max_sim, idx

# retourner la réponse selon question_list ou answer_list
def get_answer(query_vect, question_list, answer_list):
    sim_query_question_max, idx_qq = get_similarity_max(
        query_vect, question_list)
    sim_query_answer_max, idx_qr = get_similarity_max(
        query_vect, answer_list)
    if sim_query_question_max < sim_query_answer_max:
        return answer_list[idx_qq]
    else:
        return answer_list[idx_qr]

In [13]:
# différentes inferfaces à connecter avec des modules dans le web frontal
# salutation de bot
def welcome_bot():
    content = "Hello, my name is Palette, I'm the baby of mom Shanshan and dad Cancan, I can speak French, Chinese and English, what language do you speak? Please input en or zh or fr "
    print("BOT: ", content)
    # return content

# choisir la langue
def feedback_welcome(language):
    if language == 'fr':
        print("BOT : D'accord, vous avez choisir la langue française, qu'est-ce que je peux vous aider ? ")
        # return "D'accord, qu'est-ce que je peux vous aider ? "
    if language == 'zh':
        print("BOT : 好的，您选择了中文服务，请问有什么可以帮助您的呢？")
        # return "好的，请问有什么可以帮助您的呢？"
    if language == 'en':
        print("BOT : Okay, you have choisen English, what can I help you with ? ")
        # return "Okay, what can I help you with ? "

# réaction après le choix de langue
def feedback_question(language):
    if language == 'fr':
        print('maintenant vous pouvez entrer votre question')
    if language == 'zh':
        print('请您输入您想问的问题')
    if language == 'en':
        print('now you can entrer your questions.')

# trois boutons pour les différents types de questions
def feedback_type(language):
    if language == 'fr':
        print('Veillez choisir le domaine à consulter : psychology, drug, joke, others')
    if language == 'zh':
        print('请选择您想咨询的领域 ： 心理咨询（psychology）, 药品咨询(drug), 其他(others)')
    if language == 'en':
        print('Please select your type for consulting : psychology, drug, joke, others')

# fonction blague
def feed_back_joke(language, df_en, df_fr):
    if language == 'fr':
        df = df_fr
    if language == 'en':
        df = df_en
    print(df['Joke'].sample(1))
    print(df['Answer'].sample(1))
    # return df['Joke'].sample(1), df['Answer'].sample(1)

# chercher dans différentes DataFrame selon le type
def type_choix(psy_df, other_df, drug_df, df_en, df_fr, language):
    feedback_type(language)
    type_ch = input()
    print('YOU : ', type_ch)
    if type_ch == 'joke':
        feed_back_joke(language, df_en, df_fr)
    else:
        if type_ch == 'psychology':
            question_list, answer_list = get_list(psy_df)
            feedback_question(language)
            return question_list, answer_list
        if type_ch == 'others':
            question_list, answer_list = get_list(other_df)
            feedback_question(language)
            return question_list, answer_list
        if type_ch == 'drug':
            question_list, answer_list = get_list(drug_df)
            feedback_question(language)
            return question_list, answer_list

# chercher la meilleure réponse en donnant la langue et la requête
def communication(language, query, question_list, answer_list):
    # si le langage choisi n'est pas l'anglais, il faut d'abord traduire la requête en anglais
    if not language == 'en':
        query_en = translator.translate(
            query, lang_src=language, lang_tgt='en')
        
        # entrer dans le modèle sentenceBERT pour le convertir en vecteurs
        query_vect = sentence_bert_model.encode([query_en])[0]
        answer = get_answer(query_vect, question_list, answer_list)
        '''
        Dans la question_list, il y a deux types de réponses qui montrent que les réponses ne sont pas disponibles.
        ans ce cas-là, on souhaite qu'il puisse retourner une réponse unique et rédigée
        '''
        if not answer == 'No answer' or 'Unanswerable':
            answer_translate = translator.translate(
                answer, lang_src='en', lang_tgt=language)
            print(answer_translate)
        else:
            print(
                "BOT : Sorry, I don't know how to answer this question, but I will try hard to learn it.")
    else:
        query_vect = sentence_bert_model.encode([query])[0]
        answer = get_answer(query_vect, question_list, answer_list)
        if not answer == 'No answer' or 'Unanswerable':
            print(answer)
        else:
            print(
                "BOT : Sorry, I don't know how to answer this question, but I will try hard to learn it.")
    # return "Sorry, I don't know how to answer this question, but I will try hard to learn it."
    # return answer_translate

# salutation à la fin
def feedback_bye(language):
    if language == 'fr':
        print('BOT : Au revoir ! Bonne journée !')
    if language == 'zh':
        print('BOT : 再见，祝您拥有愉快的一天 ！')
    if language == 'en':
        print('BOT : Bye ! Have a nice day !')
        
# fonction finale
def chatbot():
    try:
        datapath = 'data/data_final.csv'
        data_frame = get_dataframe(datapath)
        joke_fr_df = pd.read_csv('data/joke_fr.csv', sep=';', header=0)
        joke_en_df = pd.read_csv('data/joke_en.csv', sep=';', header=0)

        psycho_frame, other_frame, drug_frame = classify_data(data_frame)

        welcome_bot()
        language = input()
        feedback_welcome(language)

        question_list, answer_list = type_choix(
            psycho_frame, other_frame, drug_frame, joke_en_df, joke_fr_df, language)

        # une boucle qui permet à donner des requetes sans arrêt
        while True:
            try:
                query = input()
                print('YOU : ', query)
                if 'bye' in query:
                    feedback_bye(language)
                    break
                else:
                    communication(language, query, question_list, answer_list)
            except (KeyboardInterrupt, EOFError, SystemExit):
                print("Sorry,I've met some problems.")
                break
    except (KeyboardInterrupt, EOFError, SystemExit):
        print("Sorry,I've met some problems.")


In [19]:
# test français
if __name__ == '__main__':
    chatbot()

BOT:  Hello, my name is Palette, I'm the baby of mom Shanshan and dad Cancan, I can speak French, Chinese and English, what language do you speak? Please input en or zh or fr 
fr
BOT : D'accord, vous avez choisir la langue française, qu'est-ce que je peux vous aider ? 
Veillez choisir le domaine à consulter : psychology, drug, joke, others
psychology
YOU :  psychology
maintenant vous pouvez entrer votre question
j'ai la solitude
YOU :  j'ai la solitude
Il est naturel de se sentir seul quand on est seul trop longtemps, et on ne peut pas échanger des idées et des sentiments authentiques avec au moins un ami ...: je ne sais pas si le sentiment que l'on a en plus de la solitude et vraiment du «vide» "..., ou" simplement "un manque (d'échanges et / ou d'affection, qui accompagnent toujours une relation d'amitié, sinon pas d'amour). 
comment faire si j'arrive pas à dormir
YOU :  comment faire si j'arrive pas à dormir
Votre sommeil est le reflet de votre état intérieur. Vous dites que vous êt

In [15]:
# test anglais
if __name__ == '__main__':
    chatbot()

BOT:  Hello, my name is Palette, I'm the baby of mom Shanshan and dad Cancan, I can speak French, Chinese and English, what language do you speak? Please input en or zh or fr 
en
BOT : Okay, you have choisen English, what can I help you with ? 
Please select your type for consulting : psychology, drug, joke, others
psychology
YOU :  psychology
now you can entrer your questions.
How can I calm my anxiety when I'm alone?
YOU :  How can I calm my anxiety when I'm alone?
It is natural to feel alone when you are alone for too long, and you cannot exchange authentic ideas and feelings with at least one friend ...: I don't know if the feeling that you have in addition to loneliness and really "emptiness" ..., or "simply" a lack (of exchanges and / or affection, which always accompany a relationship of friendship, if not no love).
how to establish a friendship relationship with a person of the opposite sex
YOU :  how to establish a friendship relationship with a person of the opposite sex
You 

In [18]:
# test chinois
if __name__ == '__main__':
    chatbot()

BOT:  Hello, my name is Palette, I'm the baby of mom Shanshan and dad Cancan, I can speak French, Chinese and English, what language do you speak? Please input en or zh or fr 
zh
BOT : 好的，您选择了中文服务，请问有什么可以帮助您的呢？
请选择您想咨询的领域 ： 心理咨询（psychology）, 药品咨询(drug), 其他(others)
psychology
YOU :  psychology
请您输入您想问的问题
我睡不着觉怎么办
YOU :  我睡不着觉怎么办
睡眠是您内在状态的反映。 您说自己很着急或处于恐慌状态，看到的东西都是黑色的...不要被这些感觉所抛弃。 您是在经历此阶段还是近期？ 谁会陪着你？ 您对自己的整体疲劳感如何？ 睡眠不足是增加疲劳感和焦虑感的一种手段。 展现自己的感受，感受在您的存在中可以为您带来光明的事物。 观察当前生活中正在发生的事情，并了解哪些事情无法满足您今天的深切需求。 通过找到更像您的道路，您可以得到支持以识别您的需求和焦虑的根源。 
如何重拾对生活的兴趣
YOU :  如何重拾对生活的兴趣
您需要盘点自己的生活。 毫无疑问，回想一下您的遥远过去（童年时代），它将使您了解自己的操作方式。 
我失去了对童年的记忆
YOU :  我失去了对童年的记忆
可能会让人忘记了童年时期的一切-这是由于多种因素，尤其是情感因素-很难回答，除了您的年龄，背景， 职业，家庭状况和可能的夫妻生活。 
我的梦有什么启示作用啦
YOU :  我的梦有什么启示作用啦
你的梦想诉说着你的愿望。 现在您已经很清楚了，要么您将其实现为现实……要么继续梦想。 预测未来不是心理学家的工作。 
bye
YOU :  bye
BOT : 再见，祝您拥有愉快的一天 ！
